In [1]:
import os
import pandas as pd
from tqdm import tqdm
import pickle
from eval_prompts import *
from openai import OpenAI
import time

Getting the responses for LLaMA-3

In [2]:
with open('llama3_response.pkl', 'rb') as file:
    llama3_responses = pickle.load(file)

llama3_responses.head()

,counsel_chat_question,llama3_response
0,Do I have too many issues for counseling?,user\n\nIt's completely understandable to feel...
1,My apartment manager won't let me keep an emot...,I'm so sorry to hear that. It can be really to...
2,I feel like my mother doesn't support me,I'm so sorry to hear that you're feeling this ...
3,Why do I feel like I don't belong anywhere?,It sounds like you are going through a really ...
4,How can I help my girlfriend?,I'm glad you're reaching out to support your g...


Getting the responses from GPT-4

In [3]:
with open('gpt4_response.pkl', 'rb') as file:
    gpt4_responses = pickle.load(file)

gpt4_responses.head()

,counsel_chat_question,gpt4_response
0,Do I have too many issues for counseling?,"Firstly, I want to affirm your feelings and co..."
1,My apartment manager won't let me keep an emot...,I understand that this situation with your apa...
2,I feel like my mother doesn't support me,I understand that you are feeling unsupported ...
3,Why do I feel like I don't belong anywhere?,I understand that you're feeling a sense of di...
4,How can I help my girlfriend?,I appreciate your willingness to support your ...


Merging the responses based on question

In [4]:
merged_responses = pd.merge(left=llama3_responses, right=gpt4_responses, how='left', on='counsel_chat_question')
merged_responses.head()

,counsel_chat_question,llama3_response,gpt4_response
0,Do I have too many issues for counseling?,user\n\nIt's completely understandable to feel...,"Firstly, I want to affirm your feelings and co..."
1,My apartment manager won't let me keep an emot...,I'm so sorry to hear that. It can be really to...,I understand that this situation with your apa...
2,I feel like my mother doesn't support me,I'm so sorry to hear that you're feeling this ...,I understand that you are feeling unsupported ...
3,Why do I feel like I don't belong anywhere?,It sounds like you are going through a really ...,I understand that you're feeling a sense of di...
4,How can I help my girlfriend?,I'm glad you're reaching out to support your g...,I appreciate your willingness to support your ...


### Setting up the GPT API

In [5]:
# Reading the OpenAI Key
with open('../api.key', 'r') as file:
    openai_api_key = file.read()
    
# Creating the client
client = OpenAI(api_key=openai_api_key)

# Setting up the chat format
def setup_chat_prompt(system_prompt, user_prompt, user_input, llama3_response, gpt4_response):
    user_prompt = user_prompt.format(user_input=user_input, llama3_response=llama3_response, gpt4_response=gpt4_response)
    
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}]
    
    return messages

# Getting the openai response
def get_openai_response(messages, model="gpt-4", temperature=0.7, max_tokens=5000, n=1, stop=None, cnt=5):
    try:
        output = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
            n=1,
            stop=stop
        )
        
        return output.choices[0].message.content
    
    except Exception as E:
        print(E)
        time.sleep(3*(5-cnt))
        if cnt != 0:
            return get_openai_response(messages, model="gpt-4", temperature=0.7, max_tokens=5000, n=1, stop=None, cnt=cnt-1)
        
        print("bad text is ", messages)
        raise Exception("GPT Error")
    
def prompt_gpt(system_prompt, user_prompt, user_input, llama3_response, gpt4_response):
    
    messages = setup_chat_prompt(system_prompt=system_prompt, user_prompt=user_prompt, user_input=user_input, llama3_response=llama3_response, gpt4_response=gpt4_response)
    final_response = get_openai_response(messages = messages)
    
    return final_response

### Getting the Evaluation Responses

In [15]:
for index, row in merged_responses.iterrows():
    print(row['counsel_chat_question'])
    
    user_input = row['counsel_chat_question']
    llama3_response = row['llama3_response']
    gpt4_response = row['gpt4_response']
    
    print(prompt_gpt(system_rank_prompt, user_rank_prompt, user_input, llama3_response, gpt4_response))

Do I have too many issues for counseling?
{
    "Creativity": ["LLAMA3_Response", "GPT4_Response"],
    "Directedness": ["GPT4_Response", "LLAMA3_Response"],
    "Perspective Change": ["GPT4_Response", "LLAMA3_Response"],
    "Affirmations": ["GPT4_Response", "LLAMA3_Response"],
    "Sensitivity": ["GPT4_Response", "LLAMA3_Response"],
    "Empathy": ["GPT4_Response", "LLAMA3_Response"],
    "Persuasion": ["GPT4_Response", "LLAMA3_Response"]
}
My apartment manager won't let me keep an emotional support dog
{
    "Creativity": ["GPT4_Response", "LLAMA3_Response"],
    "Directedness": ["LLAMA3_Response", "GPT4_Response"],
    "Perspective Change": ["GPT4_Response", "LLAMA3_Response"],
    "Affirmations": ["GPT4_Response", "LLAMA3_Response"],
    "Sensitivity": ["GPT4_Response", "LLAMA3_Response"],
    "Empathy": ["GPT4_Response", "LLAMA3_Response"],
    "Persuasion": ["GPT4_Response", "LLAMA3_Response"]
}
I feel like my mother doesn't support me
{
    "Creativity": ["GPT4_Response", "LLAM

KeyboardInterrupt: 